## 池化层

降低卷积层对位置的敏感性，同时降低对空间降采样表示的敏感性。

In [1]:
import torch
from torch import nn

### 1.最大池化层和平均池化层

与卷积层类似，汇聚层运算符由一个固定形状的窗口组成，该窗口根据其步幅大小在输入的所有区域上滑动，为固定形状窗口（有时称为池化窗口）遍历的每个位置计算一个输出。
- 池化窗口不包含参数，池运算符是确定的；
- 通常计算汇聚窗口中所有元素的最大值（maximum pooling）或平均值（average pooling）。

In [4]:
def pool2d(X, pool_size, mode='max'):
    '''2d单通道无padding无stride的池化操作，pool_size = (p_h, p_w)，mode = 'max' or 'avg' or 'min' '''
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i:i+p_h, j:j+p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i:i+p_h, j:j+p_w].mean()
            elif mode == 'min':
                Y[i, j] = X[i:i+p_h, j:j+p_w].min()
    return Y

In [5]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [6]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

### 2.填充、步幅和多个通道

- 与卷积层相似，都具有填充和步幅；
- 没有可学习的参数；
- 在每个输入通道，应用池化层，获得相应的输出通道；
- 输出通道数 = 输入通道数。

In [7]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [8]:
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

填充和步幅可以手动设定

In [9]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [10]:
pool2d = nn.MaxPool2d((2, 3), padding=(1, 1), stride=(2, 3))
pool2d(X)

tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

In [11]:
X = torch.cat((X, X+1), 1)  # cat维度不变，zip添加新维度
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [12]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

### 3.小结

- 池化层返回窗口中最大或平均值；
- 缓解卷积层会对位置的敏感性；
- 同样有窗口大小、填充、步幅，作为超参数。